In [1]:
# ------------------------------------------------------------
# Workshop 4 - Scenario 2: Cellular Automata Simulation
# Autómata Celular para Dinámica Espacial del Mercado Inmobiliario
# Estados:
# 0 = estable
# 1 = riesgoso
# 2 = ruido/caos
# ------------------------------------------------------------

import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter

# ----- CONFIG -----
GRID_SIZE = 20
ITERATIONS = 40

OUTPUT_DIR = "outputs"
FRAMES_DIR = f"{OUTPUT_DIR}/frames"
os.makedirs(FRAMES_DIR, exist_ok=True)

# ----- INITIAL GRID -----
states = [0, 1, 2]
probs = [0.70, 0.20, 0.10]
grid = np.random.choice(states, size=(GRID_SIZE, GRID_SIZE), p=probs)

history = [grid.copy()]

# ----- NEIGHBORS -----
def get_neighbors(grid, x, y):
    neighbors = []
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            nx, ny = x+dx, y+dy
            if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                neighbors.append(grid[nx, ny])
    return neighbors

# ----- UPDATE RULE -----
def update_state(cell_state, neighbors):
    # Regla 1: Riesgo se contagia
    if neighbors.count(1) > 3:
        return 1

    # Regla 2: Influencia del ruido
    if 2 in neighbors and np.random.rand() < 0.25:
        if cell_state == 0:
            return 1
        elif cell_state == 1:
            return 2
        else:
            return 1

    # Regla 3: Recuperación
    if cell_state == 1 and neighbors.count(0) >= 4:
        return 0

    return cell_state

# ----- EVOLUTION -----
for t in range(ITERATIONS):
    new_grid = grid.copy()

    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            neighbors = get_neighbors(grid, i, j)
            new_grid[i, j] = update_state(grid[i, j], neighbors)

    # Regla 4: Perturbación global cada 10 iteraciones
    if t % 10 == 0 and t > 0:
        num_shock = int(GRID_SIZE * GRID_SIZE * 0.03)
        idxs = np.random.choice(GRID_SIZE*GRID_SIZE, num_shock, replace=False)
        for k in idxs:
            x, y = divmod(k, GRID_SIZE)
            new_grid[x, y] = 2

    grid = new_grid.copy()
    history.append(grid.copy())

    # ----- Save frame -----
    plt.figure(figsize=(4,4))
    plt.imshow(grid, cmap="viridis")
    plt.title(f"Iteration {t+1}")
    plt.axis("off")
    plt.savefig(f"{FRAMES_DIR}/frame_{t+1}.png", dpi=120)
    plt.close()

# ----- Create GIF -----
gif_path = f"{OUTPUT_DIR}/automata.gif"
fig, ax = plt.subplots(figsize=(4,4))

writer = PillowWriter(fps=5)
writer.setup(fig, gif_path, dpi=100)

for step in history:
    ax.clear()
    ax.imshow(step, cmap="viridis")
    ax.axis("off")
    writer.grab_frame()

writer.finish()
plt.close()
print(f"GIF creado en: {gif_path}")

# ----- Save CSV with counts -----
history_counts = []
for idx, h in enumerate(history):
    unique, counts = np.unique(h, return_counts=True)
    data = dict(zip(unique, counts))
    history_counts.append({"iter": idx, "state0": data.get(0,0),
                           "state1": data.get(1,0),
                           "state2": data.get(2,0)})

import pandas as pd
df = pd.DataFrame(history_counts)
df.to_csv(f"{OUTPUT_DIR}/history.csv", index=False)

print("Simulación completada correctamente.")


GIF creado en: outputs/automata.gif
Simulación completada correctamente.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=736728e4-d5ba-4e9b-a46b-d07507b2f195' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>